# 5 cell 800 MHz rectangular cavity 

* [PERLE collaboration meeting cavity talk](https://indico.ijclab.in2p3.fr/event/7907/contributions/24619/attachments/18114/23933/Rimmer_ERL_cavity.pdf)
* [PERLE HOM coupler](https://accelconf.web.cern.ch/linac2022/papers/thpojo21.pdf)

Cavity angular frequency $\omega$ as a function of radius $R$
$$ \omega = 2\pi f = \frac{2.405c}{R} $$

So for a $\pi$ mode cavity the distance between two cells is 

$$ l_{\rm cell} = \frac{1}{2f} c$$

For an $n_{\rm cell}$ cavity the total cavity length is then 

$$ l_{\rm cavity} = n_{\rm cell} l_{\rm cell}$$

In [ ]:
import numpy as np

def freq_from_radius(radius_m) :
    return 2.405*3e8/radius_m/1e9/2/np.pi

def radius_from_freq(freq_ghz) : 
    return 2.405*3e8/(2*np.pi*freq_ghz*1e9)

def cell_length(freq_ghz) :
    return 1./(freq_ghz*1e9)/2*3e8

In [ ]:
rcell = radius_from_freq(0.8)
lcell = cell_length(0.8)
lcav  = 5*lcell
print(f"rcell={rcell:5.3} lcell={lcell:5.3} lcav={lcav:5.3}")

In [ ]:
import pyCavity2d
#b_sc = pyCavity2d.DomainLoader.load2d_boundary("pillbox_0.8GHz.dat")
b_sc = pyCavity2d.DomainCreator.make2d_rectangular(cavity_freqnecy_ghz=0.8,
                                                   iris_thickness=0.005,
                                                   iris_radius=0.03)
d_sc = pyCavity2d.Domain2D(b_sc,single_cell=True,maxh=0.005)
fc_sc = pyCavity2d.FieldCalculator(d_sc)
fc_sc.compute(maxit=200)

In [ ]:
fc_sc.draw(1)

In [ ]:
import pyCavity2d
import matplotlib.pyplot as _plt
import numpy as _np
b_sc = pyCavity2d.DomainCreator.make2d_rectangular(cavity_freqnecy_ghz=0.8,
                                                   iris_thickness=0.005,
                                                   iris_radius=0.03)
b_mc = pyCavity2d.DomainCreator.make_boundary_repeat(b_sc,5)

#_plt.subplot(2,1,1)
#pyCavity2d.DomainCreator.draw(b_sc)
#_plt.subplot(2,1,2)
#pyCavity2d.DomainCreator.draw(b_mc)

d_mc = pyCavity2d.Domain2D(b_mc,single_cell=True,maxh=0.005) #maxh=0.0025)
fc_mc = pyCavity2d.FieldCalculator(d_mc)
fc_mc.compute(maxit=200)

In [ ]:
fc_mc.draw(4,field="E")
fc_mc.vtk(4,field="E")

# Comparing field with results from(Bjorkquist and Rosenzweig/Serafini)

Starting with the axial accelerating field $E_z$, the radial electric field $E_r$ and azimuthal magnetic field $B_{\phi}$ are 

$$ E_r = -\frac{1}{2} r \frac{\partial E_z}{\partial z}$$

$$ B_{\phi} = \frac{1}{c^2} \frac{r}{2} \frac{\partial E_z}{\partial t}$$

In [ ]:
z, e = fc_mc.plotAxialFields(5,field="E")
z, h = fc_mc.plotAxialFields(5,field="H")

_plt.subplot(4,1,1)
#_plt.plot(z,_np.abs(e[:,0]))
_plt.plot(z,e[:,0])
_plt.ylabel("$E_z$")
_plt.subplot(4,1,2)
_plt.plot(z,e[:,1])
_plt.plot(z,-_np.gradient(e[:,0], z)/2*0.005, label="JR", ls="--")
_plt.ylabel("$E_r$")
_plt.legend()

_plt.subplot(4,1,3)
_plt.plot(z,_np.abs(_np.imag(h[:,0])))
_plt.plot(z,_np.abs(e[:,0])/2/3e8**2*(2*_np.pi*802e6)/(4*_np.pi*1e-7)*0.005, label="JR", ls="--")
_plt.ylabel("$H_{\\phi}$")

_plt.legend()
_plt.tight_layout()

In [ ]:
r = 0.005

z, e = fc_mc.plotAxialFields(5,field="E",radius=r)
z, h = fc_mc.plotAxialFields(5,field="H",radius=r)

_plt.subplot(4,1,1)
_plt.plot(z,_np.abs(e[:,0]))
_plt.ylabel("$E_z$")
_plt.subplot(4,1,2)
_plt.plot(z,e[:,1])
_plt.plot(z,-_np.gradient(e[:,0], z)/2*r, label="JR", ls="--")
_plt.ylabel("$E_r$")
_plt.legend()

_plt.subplot(4,1,3)
_plt.plot(z,_np.abs(_np.imag(h[:,0])))
_plt.plot(z,_np.abs(e[:,0])/2/3e8**2*(2*_np.pi*802e6)/(4*_np.pi*1e-7)*r, label="JR", ls="--")
_plt.ylabel("$H_{\\phi}$")

_plt.legend()
_plt.tight_layout()

### Export to superfish

In [ ]:
import pyCavity2d
import matplotlib.pyplot as _plt
import numpy as _np
b_sc = pyCavity2d.DomainCreator.make2d_rectangular(cavity_freqnecy_ghz=0.8,
                                                   iris_thickness=0.005,
                                                   iris_radius=0.03)
b_sc = pyCavity2d.DomainCreator.remove_identical(b_sc)
# pyCavity2d.DomainCreator.draw(b_sc)
pyCavity2d.DomainWriter.writeSuperfish("1Cell_800MHz.AF",b_sc)